In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Rec').getOrCreate()

In [2]:
# Importing the ALS(Alternaing Least Square) model
from pyspark.ml.recommendation import ALS
# For evaluation using the RegressionEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
# Importing data
data = spark.read.csv('Recommender_Systems/movielens_ratings.csv' , header = True , inferSchema=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [7]:
# To see the data's structure
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [8]:
# Splitting data into training and test datas
training, test = data.randomSplit([0.8,0.2])

In [13]:
# Instantiating the ALS model
als = ALS(maxIter =5,regParam =0.01,userCol='userId',itemCol = 'movieId',ratingCol='rating')

In [14]:
# Creating model by fitting data
model = als.fit(training)

In [15]:
# Predicting test data with the created model
predictions= model.transform(test)

In [16]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|     4|   1.691539|
|      1|   1.0|    28| -0.5769483|
|      6|   1.0|     1| 0.31742752|
|      6|   1.0|     2| 0.35766283|
|      6|   1.0|     6|  2.0838315|
|      3|   3.0|    14|  1.6889961|
|      3|   1.0|    21| 0.49606708|
|      3|   2.0|    22|  1.0193486|
|      3|   1.0|    29| 0.24209484|
|      5|   2.0|    26|  2.1908846|
|      4|   1.0|     5|-0.11032237|
|      4|   1.0|     9|  1.6245708|
|      4|   3.0|    10|  1.2067082|
|      4|   1.0|    24| 0.04878077|
|      2|   1.0|     3|  0.8183932|
|      2|   2.0|     7|  1.4788356|
|      2|   1.0|    17|   1.852133|
|      2|   1.0|    23|   3.734132|
|      2|   1.0|    25|   3.829813|
|      0|   1.0|     3|   1.049467|
+-------+------+------+-----------+
only showing top 20 rows



In [17]:
# Creating evaluator object
evaluator = RegressionEvaluator(metricName = 'rmse',labelCol='rating', predictionCol='prediction')

In [19]:
rmse = evaluator.evaluate(predictions)

In [21]:
print("RMSE")
rmse

RMSE


1.8727345548009537

In [29]:
# Applying filter for single user
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [30]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      6|    11|
|     10|    11|
|     21|    11|
|     32|    11|
|     35|    11|
|     36|    11|
|     78|    11|
|     80|    11|
|     81|    11|
|     82|    11|
|     99|    11|
+-------+------+



In [31]:
# Inorder to recommend for a single user
recommendations = model.transform(single_user)
# printing the recommendations
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     36|    11|  3.7239482|
|     32|    11|  3.2374756|
|     10|    11|   3.132456|
|     21|    11|  2.6210525|
|     81|    11|  1.9411128|
|     35|    11|  1.5115551|
|     78|    11|  0.9734031|
|     99|    11|  0.7383608|
|     82|    11| 0.42540535|
|     80|    11|-0.42813414|
|      6|    11|-0.54287213|
+-------+------+-----------+

